In [5]:
# 训练
# 1 参数声明 2 hog 3 svm 4 computer hog 5 label 6 train 7 pred 8 draw
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# 1 par
# 正样本个数
PosNum = 820
# 负样本个数
NegNum = 1931
# 窗体大小
winSize = (64,128)
# block块大小
blockSize = (16,16) # 105 = （64-16) 是需要滑动的距离 / 步长 = block数 再+1
# (128-16)/8 +1 纵向需要滑动的block数
# 步长
blockStride = (8,8) # 4 cell
# cell大小
cellSize = (8,8)
# bin个数9个
nBin = 9 # 9 bin 3780个维度

# 2 hog create hog 1 win 2 block 3 blockStride 4 cell 5 bin
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nBin)

# 3 svm分类器创建
svm = cv2.ml.SVM_create()

# 4 computer hog
featureNum = int(((128-16)/8+1)*((64-16)/8+1)*4*9) # 3780
# 特征数组
featureArray = np.zeros(((PosNum+NegNum),featureNum),np.float32)
# 标签数组
labelArray = np.zeros(((PosNum+NegNum),1),np.int32)

# svm 监督学习 样本 标签 svm 学习使用 image hog特征(对于svm来说真正的样本)。  
for i in range(0,PosNum):
    fileName = 'pos/'+str(i+1)+'.jpg'
    img = cv2.imread(fileName)
    # 传入图片数据 和windows窗口步长
    hist = hog.compute(img,(8,8)) # 3780维
    for j in range(0,featureNum):
        featureArray[i,j] = hist[j]
    # 作用: featureArray hog [1,:] hog1 [2,:]hog2 
    labelArray[i,0] = 1
    # 正样本 label 1
    
for i in range(0,NegNum):
    fileName = 'neg/'+str(i+1)+'.jpg'
    img = cv2.imread(fileName)
    hist = hog.compute(img,(8,8))# 3780
    for j in range(0,featureNum):
        featureArray[i+PosNum,j] = hist[j]
    labelArray[i+PosNum,0] = -1
    # 负样本 label -1

# svm属性设置
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)

# 6 train
ret = svm.train(featureArray,cv2.ml.ROW_SAMPLE,labelArray)

# 7 myHog ：《-myDetect
# myDetect-《resultArray  rho
# myHog-》detectMultiScale

# 7 检测  核心：create Hog -》 myDetect—》array-》
# resultArray-》resultArray = -1*alphaArray*supportVArray
# rho-》svm-〉svm.train

# 一行一列
alpha = np.zeros((1),np.float32)
rho = svm.getDecisionFunction(0,alpha)
print(rho)
print(alpha)
alphaArray = np.zeros((1,1),np.float32)
# 支持向量
supportVArray = np.zeros((1,featureNum),np.float32)
resultArray = np.zeros((1,featureNum),np.float32)
alphaArray[0,0] = alpha
resultArray = -1*alphaArray*supportVArray

# detect
myDetect = np.zeros((3781),np.float32)
for i in range(0,3780):
    myDetect[i] = resultArray[0,i]
myDetect[3780] = rho[0]

# rho svm （判决）
# 构建hog
myHog = cv2.HOGDescriptor()
myHog.setSVMDetector(myDetect)

# load待检测图片
imageSrc = cv2.imread('Test2.jpg',1)

# detectMultiScale实现对目标检测 (8,8) win 
# 图片内容  (8,8)windows步长  1.05缩放系数 32窗体大小
# 核心是我们的myhog对象
objs = myHog.detectMultiScale(imageSrc,0,(8,8),(32,32),1.05,2)

# 包含x y 宽高
# xy wh 三维信息 参数在最后一维
x = int(objs[0][0][0])
y = int(objs[0][0][1])
w = int(objs[0][0][2])
h = int(objs[0][0][3])

# 绘制并展示
# 图片，起始位置，终止位置，颜色，线条宽度
cv2.rectangle(imageSrc,(x,y),(x+w,y+h),(255,0,0),2)
# 图片展示
cv2.imshow('dst',imageSrc)
# 程序等待
cv2.waitKey(0)

(0.1927380867546681, array([[1.]]), array([[0]], dtype=int32))
[0.]


255

In [2]:
# 1 样本 2 训练 3 test 预测
# 1 样本
# 1.1 pos 正样本 包含所检测目标 neg 不包含obj 图片大小均为64*128
# 1.2 如何获取样本 1 网络 2 公司内部 3 自己收集
# 一个好的样本 远胜过一个 复杂的神经网络 （K w）（M）
# 1.1 网络公司 样本：1张图 1元  贵
# 1.2 网络 爬虫 自己爬 
# 1.3 公司： 很多年积累（mobileeye ADAS 99%） 红外图像 
# 1.4 自己收集 视频 100秒 30 = 3000
# 正样本：尽可能的多样  环境 干扰
# 820 pos neg 1931 1:2 1:3
# name